In [1]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
# objective: classify cannabis' dominant strain [Indica, Sativa] 
# from chemical data

In [3]:
# Gather data ---------------------------------------------------

In [37]:
# CSV Chemical Data
loc = '~/Desktop/portfolio/CannAnxi/Canna.csv'
df = pd.read_csv(loc)

In [38]:
# cleaning CSV data -- dropping columns not of interest
to_drop = [
    'OCPID',
    'Δ8-THC',
    'SampleID',
    'DateRecorded',
    'ReportType',
    'ChemicalLab',
    'H2O',
    'GeneticLab',
    'Sample',
    'SampleURL',
    'Organism',
    'OrganismURL',
    'Project',
    'ProjectURL',
    'Study',
    'StudyURL',
    'Run',
    'RunURL',
    'DatePublished',
    'Spots',
    'Bases',
    'Size',
    'Notes',
]

# drop features
df = df.drop(to_drop, axis=1)
# cleaning data -- cut rows --> only include chemical data (rows == 255)
df = df[:255]

# save strain names / use only chemical data
namesDf = df['Strain']

# reset index of df to 'Strain'
# df.set_index('Strain')


,TotalTHC,THC,Δ9-THC,THCA,THCV,TotalCBD,CBDA,CBDV,CBDVA,CBC,...,Linalool,Isopulegol,Geraniol,β-Caryophyllene,α-Humelene,Nerolidol-1,Nerolidol-2,Guaiol,CaryophylleneOxide,α-Bisabolol
Strain,,,,,,,,,,,,,,,,,,,,,
CHEM 4,193.00,193.00,0.00,203.0,NaN,0.00,0.00,NaN,NaN,NaN,...,0.37,0.0,0.0,5.48,2.03,0.0,0.00,0.00,0.0,0.87
Z7,0.00,0.00,0.00,0.0,NaN,140.00,147.00,NaN,NaN,NaN,...,0.25,0.0,0.0,2.35,1.14,0.0,0.00,0.25,0.0,0.62
ACDC/Harlequin,141.00,141.00,0.00,138.0,NaN,56.00,55.00,NaN,NaN,NaN,...,0.39,0.0,0.0,3.11,1.20,0.0,0.00,0.20,0.0,0.32
HarleyWreck,75.00,75.00,0.00,79.0,NaN,38.00,40.00,NaN,NaN,NaN,...,0.42,0.0,0.0,2.95,0.99,0.0,0.00,0.25,0.0,0.25
Sour Amnesia,222.00,222.00,0.00,222.0,NaN,0.00,0.00,NaN,NaN,NaN,...,0.28,0.0,0.0,1.42,0.58,0.0,0.00,0.00,0.0,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Og Banana Berry,159.82,159.82,10.30,155.0,0.0,1.27,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,1.36,0.00,0.0,0.00,0.00,0.0,0.00
Holy Widow,163.76,163.76,8.62,158.0,0.0,3.55,2.44,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00
Blue Berry Blast,222.81,222.81,8.12,220.0,0.0,0.00,0.00,0.0,0.0,0.0,...,0.78,0.0,0.0,3.08,0.91,0.0,0.00,0.00,0.0,0.00


In [43]:
# drop samples:
# P.A.G.S. (Name TBD)
#df = df.drop('P.A.G.S. (Name TBD)', axis=0)
# Cream Royale (Peach Gum???)
#df = df.drop('Cream Royale (Peach Gum???)', axis=0)
# Z7
df = df.drop(Z7, axis='Strain')


NameError: name 'Z7' is not defined

In [7]:
# cleaning data -- replace NaN vals with 0 
# i.e. chemical is not present in sample
df = df.fillna(0)

In [8]:
# class labels
for name in namesDf:
    print(name)

CHEM 4
Z7
ACDC/Harlequin
HarleyWreck
Sour Amnesia
Jack Herer
Critical Cheese
Sour Tangie
WildFire
Papaya
CBD Critical Mass
Moon Puppies
Master Kush
Romulan Grapefruit
Golden Pineapple
Kush Med
Blackberry Swirl
Chuy Purps
Young Jedi
Double Sour D Mint
Cannatonic
AC/DC
Blue Dream
Lemon Kush X Mountain Girl
Corazon
White Cookies
Super Gold
Black Widow
CBDiesel
U Dub
Mystery Machine
Shaolin Kush #2
Mindful Kush
Hornblower
Black Cherry Soda
Lemon Drop
Blueberry
Grape Kush
Grinspoon
Golden Cobra
Montana Silvertip
Dogwalker OG
Pineapple Express
Steven Hawking
Durbon Poison
Wedding Cake
Key Lime Cake
Quantum Kush
Black Lime Reserve
Swiss Tsu
Llama Kush
OG-78
Wifi #3
Silver Hawk
Durban Poison
Cherry 1
Critical Cure
Guava Chem
Bubblegum
Sci Fi
Gorilla Glue #4
Purple Hindu Kush
GT Dragon
Jack's Dream
Sour Apple
CBD Therapy
Rogue Blue Cheese
Jaeger OG
Jaeger Kush
Sugar Black Rose
Dawg's Waltz
Wedding Cake
Shark Shock
Purple Hindu Kush
Cherry Wine
CBD x Shark Shock
Purple Wildfire
P.A.G.S. (Name TB

In [ ]:
# leafly --> HTTPError: HTTP Error 403: Forbidden
# Cannasos success parsing -- no success extracting
#############
# https://potguide.com/strain-profiles/blue-dream/ -- success w/ ~5 strains
# 


url = 'https://www.wikileaf.com/strain/blue-dream/'

parse_html_url = urlopen(url)
html_page = parse_html_url.read()
parse_html_url.close()

strain = BeautifulSoup(html_page, "html.parser")

vals = strain.findAll("div", {"class":"strain-description"})
#val = val.get_text()
for val in vals:
    val = val.get_text().lower()
    #print(val)
    if val.find("indica") != -1:
        print("indica")
        break
    elif val.find("sativa") != -1:
        print("sativa")
        break
    else:
        pass

In [ ]:
# Web Scrape class
ind = [] # indica
sat = [] # sativa
hyb = [] # hybrid
nf = [] # not found
for name in namesDf:
    url = 'https://www.wikileaf.com/strain/'
    url = url + str(name).replace(' ', '-').replace('"', '').replace("'").lower()
    url = url + '/'
    print(url)
    try:
        parse_html_url = urlopen(url)
        html_page = parse_html_url.read()
        parse_html_url.close()

        strain = BeautifulSoup(html_page, "html.parser")

        vals = strain.findAll("div", {"class":"strain-description"})
        #val = val.get_text()
        for val in vals:
            val = val.get_text().lower()
            #print(val)
            if val.find("indica") != -1:
                print("indica")
                ind.append(name)
                break
            elif val.find("sativa") != -1:
                sat.append(name)
                print("sativa")
                break
            elif val.find("hybrid") != -1:
                print("hybrid")
                hyb.append(name)
                break    
            else:
                nf.append(name)
                pass
        
    except:
        print(name, 'not found')
        pass


In [ ]:
print('indica')
display(ind)# indica
print('sativa')
display(sat) # sativa
print('hybrid')
display(hyb)
print('not found')
display(nf)